In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.stats import zscore
from scipy.stats.mstats import winsorize

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
# from sklearn import datasets

In [2]:
# titanic = sns.load_dataset("titanic")
# titanic

In [3]:
train = pd.read_csv(r'D:\Dataset\sklearn\Titanic-ml\train.csv')
test = pd.read_csv(r'D:\Dataset\sklearn\Titanic-ml\test.csv')

In [4]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df = pd.concat([train,test],sort = False)

In [6]:
test.shape

(418, 11)

In [7]:
train.shape

(891, 12)

In [8]:
df.shape

(1309, 12)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [10]:
df.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [11]:
df = df.drop('Cabin',axis = 1)

In [12]:
#before Age imputer is = 177
si = SimpleImputer(strategy='mean')
df['Age'] = si.fit_transform(df[['Age']])

In [13]:
#before Embarked imputer is = 2
si = SimpleImputer(strategy='most_frequent')
df['Embarked'] = si.fit_transform(df[['Embarked']]).flatten()

In [14]:
df = df.drop(['Name','Ticket'],axis = 1)

In [15]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,male,22.000000,1,0,7.2500,S
1,2,1.0,1,female,38.000000,1,0,71.2833,C
2,3,1.0,3,female,26.000000,0,0,7.9250,S
3,4,1.0,1,female,35.000000,1,0,53.1000,S
4,5,0.0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,male,29.881138,0,0,8.0500,S
414,1306,NaN,1,female,39.000000,0,0,108.9000,C
415,1307,NaN,3,male,38.500000,0,0,7.2500,S
416,1308,NaN,3,male,29.881138,0,0,8.0500,S


### Outliar Handle

In [16]:
# from scipy.stats import zscore
threshold = 3
numerical_features = ['Fare','Age']
z_scores = np.abs(zscore(df[numerical_features]))

In [17]:
print(z_scores)

     Fare       Age
0     NaN  0.611972
1     NaN  0.630431
2     NaN  0.301371
3     NaN  0.397481
4     NaN  0.397481
..    ...       ...
413   NaN  0.000000
414   NaN  0.708081
415   NaN  0.669256
416   NaN  0.000000
417   NaN  0.000000

[1309 rows x 2 columns]


In [18]:
outliears = np.where(z_scores > threshold)
outliears

(array([ 96, 116, 493, 630, 672, 745, 851, 987], dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64))

In [19]:
from scipy.stats.mstats import winsorize

df['Age']=winsorize(df['Age'],limits=[0.15,0.15])
df['Fare']=winsorize(df['Fare'],limits=[0.15,0.15])

In [20]:
# from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

df['Sex'] = oe.fit_transform(df[['Sex']])
df['Embarked'] = oe.fit_transform(df[['Embarked']])


In [21]:
df['Survived'] = si.fit_transform(df[['Survived']])

In [22]:
df['Survived'] = oe.fit_transform(df[['Survived']])

In [23]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,1.0,22.000000,1,0,7.7500,2.0
1,2,1.0,1,0.0,38.000000,1,0,57.0000,0.0
2,3,1.0,3,0.0,26.000000,0,0,7.9250,2.0
3,4,1.0,1,0.0,35.000000,1,0,53.1000,2.0
4,5,0.0,3,1.0,35.000000,0,0,8.0500,2.0
...,...,...,...,...,...,...,...,...,...
413,1305,0.0,3,1.0,29.881138,0,0,8.0500,2.0
414,1306,0.0,1,0.0,39.000000,0,0,57.0000,0.0
415,1307,0.0,3,1.0,38.500000,0,0,7.7500,2.0
416,1308,0.0,3,1.0,29.881138,0,0,8.0500,2.0


In [24]:
 X = df.drop(['Survived'],axis = 1)
 y = df['Survived']    

In [25]:
y

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
413    0.0
414    0.0
415    0.0
416    0.0
417    0.0
Name: Survived, Length: 1309, dtype: float64

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [27]:
# from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators= 100, learning_rate = 1, random_state = 0)
ab.fit(x_train, y_train)

C:\Users\Excel\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=100, random_state=0)

In [28]:
pred = ab.predict(x_test)

In [29]:
accuracy_score(pred, y_test)

0.8320610687022901